## Importing libs

In [14]:
import numpy as np
import torch
from google.colab import drive
import xml
import os
import zipfile 

## Linking kaggle dataset to drive 

In [23]:
!pip install kaggle

In [24]:
drive.mount('/content/gdrive')
os.environ['KAGGLE_CONFIG_DIR'] = '/content/gdrive/My Drive/Kaggle'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [25]:
%cd /content/gdrive/My\ Drive/Kaggle
%pwd

/content/gdrive/My Drive/Kaggle


'/content/gdrive/My Drive/Kaggle'

In [26]:
!rm -r Safety\ Helmet\ Detection

rm: cannot remove 'Safety Helmet Detection': No such file or directory


In [27]:
!kaggle datasets download -d andrewmvd/hard-hat-detection

 99% 1.21G/1.22G [00:17<00:00, 83.4MB/s]
100% 1.22G/1.22G [00:18<00:00, 71.7MB/s]


In [28]:
!unzip \*.zip 

Streaming output truncated to the last 5000 lines.
  inflating: images/hard_hat_workers1.png  
  inflating: images/hard_hat_workers10.png  
  inflating: images/hard_hat_workers100.png  
  inflating: images/hard_hat_workers1000.png  
  inflating: images/hard_hat_workers1001.png  
  inflating: images/hard_hat_workers1002.png  
  inflating: images/hard_hat_workers1003.png  
  inflating: images/hard_hat_workers1004.png  
  inflating: images/hard_hat_workers1005.png  
  inflating: images/hard_hat_workers1006.png  
  inflating: images/hard_hat_workers1007.png  
  inflating: images/hard_hat_workers1008.png  
  inflating: images/hard_hat_workers1009.png  
  inflating: images/hard_hat_workers101.png  
  inflating: images/hard_hat_workers1010.png  
  inflating: images/hard_hat_workers1011.png  
  inflating: images/hard_hat_workers1012.png  
  inflating: images/hard_hat_workers1013.png  
  inflating: images/hard_hat_workers1014.png  
  inflating: images/hard_hat_workers1015.png  
  inflating: ima

## Splitting data 

In [29]:
# Creating directories for splitting purpose
%cd /content/gdrive/My Drive/Kaggle
if os.path.exists('Safety Helmet Detection'):
  print('Directories already exist')
else:
  os.mkdir('Safety Helmet Detection')
  os.mkdir('Safety Helmet Detection/training')
  os.mkdir('Safety Helmet Detection/validation')

/content/gdrive/My Drive/Kaggle


In [30]:
df = os.listdir('images/')
data_len = len(df)
SPLIT_SIZE_ = int(0.8*data_len)

np.random.shuffle(df)

train_data = df[:SPLIT_SIZE_]
val_data = df[SPLIT_SIZE_:]

In [31]:
import shutil

imgs_path = '/content/gdrive/My Drive/Kaggle/images/'
annot_path = '/content/gdrive/My Drive/Kaggle/annotations/'

for img in train_data:

  img_src = os.path.join(imgs_path, img)
  anot_src = os.path.join(annot_path, img.split('.')[0] + '.xml')
  shutil.copy(img_src,'/content/gdrive/My Drive/Kaggle/Safety Helmet Detection/training')
  shutil.copy(anot_src,'/content/gdrive/My Drive/Kaggle/Safety Helmet Detection/training')

for img in val_data:

  img_src = os.path.join(imgs_path, img)
  anot_src = os.path.join(annot_path, img.split('.')[0] + '.xml')
  shutil.copy(img_src,'/content/gdrive/My Drive/Kaggle/Safety Helmet Detection/validation')
  shutil.copy(anot_src,'/content/gdrive/My Drive/Kaggle/Safety Helmet Detection/validation')

In [32]:
# Insepcting directories
%cd 'Safety Helmet Detection/training'
%ls

/content/gdrive/My Drive/Kaggle/Safety Helmet Detection/training
hard_hat_workers0.png     hard_hat_workers2469.xml  hard_hat_workers3965.png
hard_hat_workers0.xml     hard_hat_workers2470.png  hard_hat_workers3965.xml
hard_hat_workers1000.png  hard_hat_workers2470.xml  hard_hat_workers3967.png
hard_hat_workers1000.xml  hard_hat_workers2471.png  hard_hat_workers3967.xml
hard_hat_workers1001.png  hard_hat_workers2471.xml  hard_hat_workers3968.png
hard_hat_workers1001.xml  hard_hat_workers2472.png  hard_hat_workers3968.xml
hard_hat_workers1002.png  hard_hat_workers2472.xml  hard_hat_workers3969.png
hard_hat_workers1002.xml  hard_hat_workers2473.png  hard_hat_workers3969.xml
hard_hat_workers1003.png  hard_hat_workers2473.xml  hard_hat_workers3970.png
hard_hat_workers1003.xml  hard_hat_workers2474.png  hard_hat_workers3970.xml
hard_hat_workers1004.png  hard_hat_workers2474.xml  hard_hat_workers3971.png
hard_hat_workers1004.xml  hard_hat_workers2475.png  hard_hat_workers3971.xml
hard_hat_wo

## Converting annotations to YOLO_v5 format

In [33]:
import glob
import pickle
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join

dirs = ['training', 'validation']
classes = ['helmet', 'person', 'head']

def getImagesInDir(dir_path):
   image_list = []
   for filename in glob.glob(dir_path + '/*.png'):
       image_list.append(filename)
   return image_list

def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def convert_annotation(dir_path, output_path, image_path):
    basename = os.path.basename(image_path)
    basename_no_ext = os.path.splitext(basename)[0]

    in_file = open(dir_path + '/' + basename_no_ext + '.xml')
    out_file = open(output_path + basename_no_ext + '.txt', 'w')

    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)
    for obj in root.iter('object'):
       difficult = obj.find('difficult').text
       cls = obj.find('name').text
       if cls not in classes or int(difficult)==1:
           continue
       cls_id = classes.index(cls)
       xmlbox = obj.find('bndbox')
       
       b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text),float(xmlbox.find('ymin').text),float(xmlbox.find('ymax').text))
       bb = convert((w,h), b)
       out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

cwd = '/content/gdrive/My Drive/Kaggle/Safety Helmet Detection'

for dir_path in dirs:
    full_dir_path = os.path.join(cwd, dir_path)
    output_path = full_dir_path +'/yolo_compiled_annot'  

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    image_paths = getImagesInDir(full_dir_path)
    print('Len of image_paths', len(image_paths))

    list_file = open(full_dir_path + '.txt', 'w')

    for image_path in image_paths:
        list_file.write(image_path + '\n')     
        convert_annotation(full_dir_path, output_path, image_path)
    
    list_file.close()

    print("Finished processing: " + dir_path)

Len of image_paths 4000
Finished processing: training
Len of image_paths 1000
Finished processing: validation
